In [27]:
import pandas as pd
import numpy as np

In [28]:
combined_df=pd.read_csv("file.csv")

In [29]:
row_count, column_count = combined_df.shape
print("Row count:", row_count)
print("Column count:", column_count)


Row count: 423385
Column count: 73


In [30]:
label_distribution = combined_df['Label'].value_counts()
print("\nLabel Distribution:")
print(label_distribution)


Label Distribution:
Label
VOIP             410848
P2P                6769
CHAT               3609
STREAMING          1663
EMAIL               333
FILE TRANSFER       163
Name: count, dtype: int64


In [31]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, SpatialDropout1D,Dropout
from keras.preprocessing.text import Tokenizer
from keras.utils.np_utils import to_categorical

In [32]:
# Assuming combined_df is your DataFrame containing the data
voip_packets = combined_df[combined_df['Label'] == 'VOIP'].head(1000)
p2p_packets = combined_df[combined_df['Label'] == 'P2P'].head(1000)

remaining_packets = combined_df[(combined_df['Label'] != 'VOIP') & (combined_df['Label'] != 'P2P') ]

# Concatenate the first 2000 VOIP packets, 2000 P2P packets, and the remaining packets
final_dataset = pd.concat([voip_packets, p2p_packets, remaining_packets], ignore_index=True)
print(final_dataset.shape)


(9768, 73)


In [33]:
unwanted_columns = ['frame.len','tcp.flags.syn','tcp.flags.ack','tcp.flags.push', 'tcp.flags.reset',\
                   'data.len']  # Add all unwanted column names here
# Remove unwanted columns from the DataFrame
filtered_df = final_dataset.drop(columns=unwanted_columns)

# Display the filtered DataFrame
print(filtered_df)


      ip.proto  tcp.srcport  tcp.dstport  udp.srcport  udp.dstport  tcp.seq   
0           17          NaN          NaN      28156.0         53.0      NaN  \
1           17          NaN          NaN      28156.0         53.0      NaN   
2           17          NaN          NaN      28156.0         53.0      NaN   
3           17          NaN          NaN         53.0      28156.0      NaN   
4           17          NaN          NaN         53.0      28156.0      NaN   
...        ...          ...          ...          ...          ...      ...   
9763        17          NaN          NaN        443.0      51753.0      NaN   
9764        17          NaN          NaN      51753.0        443.0      NaN   
9765        17          NaN          NaN      34134.0         53.0      NaN   
9766        17          NaN          NaN      34134.0         53.0      NaN   
9767        17          NaN          NaN         53.0      34134.0      NaN   

      frame.time_epoch  tcp.stream  ip.flags.df  fr

In [34]:
# Create a LabelEncoder object
label_encoder = LabelEncoder()

# Fit and transform the 'Label' column
filtered_df['Label'] = label_encoder.fit_transform(filtered_df['Label'])

 # Display the encoded labels
print("Encoded Labels:")
print(filtered_df['Label'].value_counts())


Encoded Labels:
Label
0    3609
5    2000
3    2000
4    1663
1     333
2     163
Name: count, dtype: int64


In [35]:
# Mapping original labels to encoded values
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print(label_mapping)

{'CHAT': 0, 'EMAIL': 1, 'FILE TRANSFER': 2, 'P2P': 3, 'STREAMING': 4, 'VOIP': 5}


In [36]:
X = filtered_df.drop(columns=['Label'])
y = filtered_df['Label']


In [37]:
encoder = LabelEncoder()
y = encoder.fit_transform(y)
num_classes = len(encoder.classes_)


In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)
print(y_train.shape)

(7814, 66)
(7814,)


In [39]:
from sklearn.impute import SimpleImputer

# Impute missing values in the features
imputer = SimpleImputer(strategy='mean')
X_train = imputer.fit_transform(X_train)
X_test= imputer.transform(X_test)

In [40]:
x_training_data = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))
print(x_training_data.shape)


(7814, 28, 1)


In [41]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, SimpleRNN, Dropout

# Define input shape based on your data
input_shape = (x_training_data.shape[1], 1)  # Shape of your input data assuming it's sequential

# Define the number of classes for the output
num_classes = 6  # Assuming you have 6 classes for the output

# Create the combined RNN-CNN model
model = Sequential([
    # CNN layers
    Conv1D(32, 3, activation='relu', input_shape=input_shape),
    MaxPooling1D(2),
    Conv1D(64, 3, activation='relu'),
    MaxPooling1D(2),
    #Flatten(),

    # RNN layers
    SimpleRNN(units=64, return_sequences=True),
    Dropout(0.2),
    SimpleRNN(units=64),
    Dropout(0.2),

    # Output layer
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Print the model summary
model.summary()

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=24, validation_data=(X_test, y_test))


Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_13 (Conv1D)          (None, 26, 32)            128       
                                                                 
 max_pooling1d_13 (MaxPoolin  (None, 13, 32)           0         
 g1D)                                                            
                                                                 
 conv1d_14 (Conv1D)          (None, 11, 64)            6208      
                                                                 
 max_pooling1d_14 (MaxPoolin  (None, 5, 64)            0         
 g1D)                                                            
                                                                 
 simple_rnn_17 (SimpleRNN)   (None, 5, 64)             8256      
                                                                 
 dropout_17 (Dropout)        (None, 5, 64)            

Epoch 46/100
326/326 [==============================] - 2s 5ms/step - loss: 0.4478 - accuracy: 0.8467 - val_loss: 0.4504 - val_accuracy: 0.8439
Epoch 47/100
326/326 [==============================] - 2s 7ms/step - loss: 0.4317 - accuracy: 0.8540 - val_loss: 0.4532 - val_accuracy: 0.8454
Epoch 48/100
326/326 [==============================] - 2s 7ms/step - loss: 0.4416 - accuracy: 0.8482 - val_loss: 0.4740 - val_accuracy: 0.8408
Epoch 49/100
326/326 [==============================] - 2s 7ms/step - loss: 0.4393 - accuracy: 0.8517 - val_loss: 0.4969 - val_accuracy: 0.8373
Epoch 50/100
326/326 [==============================] - 2s 7ms/step - loss: 0.4355 - accuracy: 0.8519 - val_loss: 0.4541 - val_accuracy: 0.8419
Epoch 51/100
326/326 [==============================] - 2s 7ms/step - loss: 0.4235 - accuracy: 0.8517 - val_loss: 0.4594 - val_accuracy: 0.8424
Epoch 52/100
326/326 [==============================] - 2s 7ms/step - loss: 0.4291 - accuracy: 0.8528 - val_loss: 0.4438 - val_accuracy:

In [42]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Define input shape based on your data
input_shape = (x_training_data.shape[1], 1)  # Shape of your input data

# Define the number of classes for the output
num_classes = 6  # Assuming you have 6 classes for the output

# Create the RNN model
rnn_model = Sequential([
    SimpleRNN(units=64, return_sequences=True, input_shape=input_shape),
    Dropout(0.2),
    SimpleRNN(units=64, return_sequences=True),
    Dropout(0.2),
    SimpleRNN(units=64),
    Dropout(0.2),
    Dense(units=num_classes, activation='softmax')
])

# Compile the model
rnn_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Print the model summary
rnn_model.summary()

# Train the model with early stopping
history = rnn_model.fit(X_train, y_train, epochs=75, batch_size=24,
                        validation_data=(X_test, y_test), callbacks=[early_stopping])


Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_19 (SimpleRNN)   (None, 28, 64)            4224      
                                                                 
 dropout_19 (Dropout)        (None, 28, 64)            0         
                                                                 
 simple_rnn_20 (SimpleRNN)   (None, 28, 64)            8256      
                                                                 
 dropout_20 (Dropout)        (None, 28, 64)            0         
                                                                 
 simple_rnn_21 (SimpleRNN)   (None, 64)                8256      
                                                                 
 dropout_21 (Dropout)        (None, 64)                0         
                                                                 
 dense_8 (Dense)             (None, 6)                

In [43]:
# Evaluate the model
score = model.evaluate(X_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.3264293968677521
Test accuracy: 0.8935517072677612


In [44]:
# Evaluate the model
score = rnn_model.evaluate(X_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.4153718054294586
Test accuracy: 0.8224155306816101


Accuracy: 1.0
